<a href="https://colab.research.google.com/github/ordevoir/Python/blob/main/10_(adv)_Files_Dirs_Scopes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с файлами и каталогами  

## Путь к интерпретатору

In [ ]:
import sys
print(sys.executable)

c:\Users\wernadsky\miniconda3\envs\tf\python.exe


## Путь к рабочему каталогу

In [ ]:
import os
work_dir = os.path.abspath("")
print(work_dir)

c:\Users\wernadsky\Documents\GitHub


In [ ]:
import os
work_dir = os.getcwd()           # возвращает путь к рабочему каталогу
print(work_dir)

c:\Users\wernadsky\Documents\GitHub


## Путь к исполняемому файлу

### Для файлов `*.ipynb`

In [ ]:
file_path = globals()['_dh'][0]     # путь к исполняемому файлу
print(str(file_path))

c:\Users\wernadsky\Documents\GitHub\Python


### Для файлов `*.py`

In [ ]:
# from os.path import abspath, dirname
# path = abspath(__file__)      # возвращает полное имя текущего файла
# location = dirname(path)      # извлекает путь из полного имени файла

### Универсальная функция

В jupyter не определена переменная `__file__`, но определена функция `get_ipython()`, в отличие от файлов, запускаемых в режиме терминала. Это обстоятельство позволяет выяснить, запускается ли фрагмент кода в jupyter.

In [ ]:
def is_running_in_jupyter():
    try:
        __file__
        return False
    except NameError:
        return True

is_running_in_jupyter()

True

Исходя из этого можно написать универсальную функцию, возвращающую путь к исполняемому файлу:

In [ ]:
def get_abs_path():
    if is_running_in_jupyter():
        path = globals()['_dh'][0]
    else:
        from os.path import abspath
        path = abspath(__file__)
    return str(path)

get_abs_path()

'c:\\Users\\wernadsky\\Documents\\GitHub\\Python'

> Пути к исполняемому файлу и рабочий каталог могут не совпадать. В общем случае рабочий каталог можно менять, в то время как расположение исполняемого файла будет прежним:

In [ ]:
import os

origin_wd = os.getcwd()                 # текущий рабочий каталог
os.chdir("..")                          # сменим рабочий каталог
file_path = get_abs_path()              # путь к исполняемому файлу
work_dir = os.getcwd()                  # путь к рабочему каталогу
os.chdir(origin_wd)                     # возврат к исходному каталогу
print(f"{file_path = }\n{work_dir =  }")

file_path = 'c:\\Users\\wernadsky\\Documents\\GitHub\\Python'
work_dir =  'c:\\Users\\wernadsky\\Documents\\GitHub'


По умолчанию текущим рабочим каталогом будет каталог, из которого запускается файл в коммандной сторке, а не каталог, в котором расположен исполняемый файл.

Кроме того, пути поиска файлов не имеют никакого отношения к путям поиска модулей.

## Прочие функции


`os.mkdir('folder')` создает новую директорию

`os.makedirs('another1/another2/another3')` – сделать несколько вложенных папок.
`os.rename("text.txt", "renamed-text.txt")` – переименование файла или папки.

Переместить (с заменой) файл в другой каталог (переименовать полный путь):
`os.replace("renamed-text.txt", "folder/renamed-text.txt")`.

`os.path.isdir('folder')` возвращает `True`, если каталог `folder` существует.

`os.path.exists()` возвращает `True`, если файл/каталог с заданным именем существует.

`os.path.dirname()` отбрасывает имя файла и возвращает путь.

`os.path.basename()` отбрасывает путь и оставляет только имя файла/каталога. Например, `'foo/bar'` превратится в `'bar'`, а `'foo/bar/'` превратится в `''`.

`os.path.splitext()` позволяет отделить расширение от имени файла

In [ ]:
from os.path import dirname, basename, splitext

path = "files/images/picture.png"
location = dirname(path)
filename, extension = splitext(basename(path))

location, filename, extension

('files/images', 'picture', '.png')

In [ ]:
basename(location)

'images'

## Чтение файла и запись

In [ ]:
# Чтение
with open('Python\\file', 'r') as file:
    file.read()         # получить содержимое файла
    file.readline()     # построчное стение
    file.readlines()    # получить список строк
    for line in file:   # перебор строк файла
        print(line)

# Запись
with open('Python\\file', 'w') as file:
    file.write('some text')            # запись в файл
    file.writelines(['some', 'text'])  # запись списка строк в файл

Переменная `file` является объектому класса _io.TextIOWrapper. С другими методами этого класса можно ознакомиться <a href="https://www.w3schools.com/python/python_ref_file.asp">здесь</a>

## Скачивание файла

### `requests`

Один из сопособов – воспользоваться модулем `requests`. Функция `get()` возвращает объект класса `Response`. Поле `status_code` содержит статус ответа, а поле `content` – содержимое ответа в байтовом представлении (объект `bytes`).

Приведенная функция `download()` по умолчанию загружает файл в рабочий каталог, но также можно задать путь (абсолютный или относительный).

In [20]:
import requests
from os import mkdir
from os.path import basename, isdir

def download(url, location='.'):
    response = requests.get(url)
    if response.status_code != 200:
        raise IOError(f"response statis code is {response.status_code}")

    if not isdir(location):
        mkdir(location)

    path = location + '/' + basename(url)
    with open(path, "wb") as file:
        file.write(response.content)

    print(f'The file is successfully downloaded with path: \n{path}')

In [21]:
url = "https://raw.githubusercontent.com/ordevoir/Python/main/images/example_img.png"
download(url)

The file is successfully downloaded with path: 
./example_img.png


### `urlretrieve()`

Другой сопосб – использование функции `urlretrieve()` из модуля `urllib.request`. Функция `urlretrieve` копирует объект на диск. Первый аргумент – **url** объекта. Если второй аргумент (`filename`) не задан, то объект будет сохранен во временных файлах. Можно задать `filename`, и тогда объект будет сохранен в соответствующем месте с соответствующим именем. Функция `urlretrieve` возвращает кортеж `(filename, headers)` – имя файла и заголовки запроса.

Важное отличие следующей функции `download()` от предыдущей в том, что здесь файл не будет перезаписан, если он уже существует.

In [22]:
from os.path import basename, exists
from urllib.request import urlretrieve

def download(url, location='.'):
    path = location + '/' + basename(url)
    if not exists(path):
        local_filename, headers = urlretrieve(url, path)
        print('Downloaded ' + path)
    else:
        print('file ' + path + ' already exists')

### Скачивание модуля из репозитория

Разместим файл модуля в той же директории, где и исполняемый файл:

In [23]:
url = "https://raw.githubusercontent.com/ordevoir/Miscellaneous/master/tools.py"

path = get_abs_path()
download(url, path)

file /content/tools.py already exists


# Области видимости  

Часто глобальные переменные используются в качестве флагов логических переменных, которые могут играть роль режимов или настроек программы.



In [ ]:
count = True        # глобальная переменная
def example():
    if count:       # использование переменной
        print('variable count is True!')

example()

variable count is True!



## Правило **LEGB**

Когда производится обращение к переменной, интерпертатор ищет ее в следующем порядке:
- в локальной области видимости самой функции (**Local** Scope)
- в локальной области видимости объемлющей функции (**Enclosing** Scope)
- в глобальной области видимости модуля (**Global** Scope)
- во встроенной области видимости (**Built-in** Scope)

Последная область относится к встроенному модулю с именем `builtins` стандартной библиотеки. Имена этого модуля представляют собой встроенные функции, такие как `print()`, `range()`, `len()`, `list()`, `object()` и т.д. Подробнее [здесь](https://docs.python.org/3/library/functions.html).


## `global` и `nonlocal`

Для изменение значения неизменяемой глобальной переменной внутри функции, или присвоения ей нового значения, необходимо в теле функции объявить имена переменных глобальными при помощи оператора `global`.

В коде ниже содержимое глобальных перменных `a` и `b` изменится внутри функции `example()`. Перменная `c` не будет изменена: инструкция `c = 10` создаст локальную переменнус `c`.

In [ ]:
a, b, c = 1, 2, 3   # глобальная переменная
def example1():
    global a, b     # без этой строки не сработает
    a += 1          # изменение глобальной переменной
    b = 5           # присвоение нового значения глобальной переменной
    c = 10          # создание локальной переменной внутри функции

example1()
print(f"{a = }, {b = }, {c = }")

a = 2, b = 5, c = 3


Если глобальной переменной указанной в `global` не существует, она будет создана, однако при этом необходимо произвести инициализацию значения.

In [ ]:
def example2():
    global x    # объявление глобальной переменной x
    x = 15      # инициализация глобальной переменной

def f():
    global x    # использовать глобальную переменную x
    x += 15     # изменение глобальной переменно внутри другой функции

In [ ]:
example2()      # создание и инициализация глобальной переменной x внутри функции
print(x)

f()             # изменение глобальной переменно внутри другой функции
print(x)

x += 10         # изменение глобальной переменной за пределами функции
print(x)

15
30
40


> Оператор `nonlocal`, в отличие от `global` сделат переменную видимой не в глобальной области, а лишь в локальной области видимости объемлющей функции (Enclosing scope).

## `globals()` и `locals()`

Функция `globals()` возвращает словарь, содержащий глобальные переменные.

Функция `locals()` возвращает словарь, содержащий имена в локальной области видимости, внутри которой вызывается функция. Непосретственно список с именами локальных переменных возвращает функция `dir()`.

> Словарь, возвращаемый функцией `locals()` при вызове из глобальной области видимости, будет совпадать со словарем, возвращаемым функцией `globals()`.

In [ ]:
def f(x):
    a = 2
    print(f"Function's local variables: {locals().keys()}")
    print(f"Function's local variables: {dir()}")
    print(f"Global variables: {globals().keys()}")

c = 3

f(c)

locals() is globals()

Function's local variables: dict_keys(['x', 'a'])
Function's local variables: ['a', 'x']
Global variables: dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_1', '_i2', '_2', '_i3', '_3', '_i4', '_4', '_i5', '_5', '_i6', '_6', '_i7', '_i8', '_i9', 'f', '_9', '_i10', '_10', '_i11', 'c', '_i12', '_12', '_i13', '_13', '_i14', '_14', '_i15', '_15', '_i16', '_16', '_i17', '_17', '_i18', '_18', '_i19', '_19', '_i20', '_20', '_i21', '_21', '_i22', '_22', '_i23', '_i24', '_24', '_i25', '_25', '_i26', '_26', '_i27', '_27', '_i28', '_i29', '_i30', 'L', '_i31', '_31', '_i32', '_i33', '_i34', 'Rectangle', 's2', 's3', '_i35', '_i36', '_i37', '_i38', '_i39', '_39', '_i40', '_i41', '_41', '_i42'])


True

>Следует иметь в виду, что словарь, возвращаемый при вызове `globals()` в модуле, не будет содержать глобальные переменные: в словаре будут содержаться только глобальные имена самого модуля.

## Получение имен переменных в виде строки

### Using `globals()`

In [3]:
def get_name_of_variable(var):
    g = globals()
    keys = g.keys()
    for key in keys:
        if globals()[key] is var and not key.startswith('_'):
            return key

In [4]:
some_variable = 1
get_name_of_variable(some_variable)

'some_variable'

### Using f-string

In [7]:
f"{some_variable=}".split("=")[0]

'some_variable'

## Замыкания (closure)

Замыкание – это особый вид функции. Она определена в теле другой функции и создаётся каждый раз во время её выполнения. Синтаксически это выглядит как функция, находящаяся целиком в теле другой функции. При этом вложенная внутренняя функция содержит ссылки на локальные переменные внешней функции (для внутренней функции это область видимости Enclosing Scope). Каждый раз при выполнении внешней функции происходит создание нового экземпляра внутренней функции, с новыми ссылками на переменные внешней функции. Поэтому замыкания можно определить как функции с расширенной областью видииости, которая охватывает все неглобальные переменные, на которые есть ссылки в теле функции, хотя они в нем не определены.


Ссылки на переменные внешней функции действительны внутри вложенной функции до тех пор, пока работает вложенная функция, даже если внешняя функция закончила работу, и переменные вышли из области видимости. <br>
Замыкание связывает код функции с её лексическим окружением (местом, в котором она определена в коде). Лексические переменные замыкания отличаются от глобальных переменных тем, что они не занимают глобальное пространство имён. От переменных в объектах они отличаются тем, что привязаны к функциям, а не объектам.

In [ ]:
def f(a):
    b = 2
    def g(c=10):
        """ Функция - замыкание """
        print(a*b, b**c)
    return g

h = f(3)
h(5)

6 32


![](https://github.com/ordevoir/Python/blob/main/images/closure.jpg?raw=1)

В данном примере в теле внешней функции `f` определена внутренняя функция `g`, которая для своей работы ссылается на переменные `a` и `b` области видимости Enclosing Scope - локальная область видимости внешней функции. При вызове функции ее локальные переменные существуют только в процессе выполнения функции, после чего их убирает сборщик мусора. Интуитивно кажется, что после выполнения функции `f` переменные `a` и `b` должны были бы уничтожиться. Однако, внешняя функции возвращает внутреннюю фунцию `g`, которая сохраняется в перменной `h`, в результате чего она появляется в глобальной области видимости (как и функция `f`). Так как `h` ссылается на функцию `g`, определенную в ходе выполнения функции `f`, а функция `g` в свою очередь ссылается на локальные переменные фунции `f`, то после выполнения фунции `f` объекты, на которые ссылались соответствующие локальные переменные не уничтожаются, так как на них продолжает ссылаться функция `g`.

Объекты замыкания содержатся в поле `__closure__` объекта функции, который представляет собой кортеж объектов класса `cell`. Для того, чтобы получть доступ непосредственно к объекту замыкания, нужно обратиться к полю `cell_contents` соответствующего объекта класса `cell`:

In [ ]:
print(type(h.__closure__), type(h.__closure__[0]))
print(h.__closure__[0].cell_contents, h.__closure__[1].cell_contents)

<class 'tuple'> <class 'cell'>
3 2


`a` и `b` – локальные переменные функции `f`, так как их инициализация происходит в теле функции `f`. Внутри `h` переменные `a` и `b` являются свободными, т.е. переменные не связаны с ее локальной областью видимости.

Инспекция возвращенного объекта `h` показывает, что Python хранит имена локальных и свободных переменных в атрибуте `__code__` (объект класса `code`), который представляет собой откомпилированное тело функции. Каждому элементу кортежа `h.__closure__` соответствует имя в `h.__code__.co_freevars`, в котором содержатся имена свободных переменных. В `h.__code__.co_namevars` содержатся имена локальных переменных.

In [ ]:
print(type(h.__code__))
print(h.__code__.co_varnames, h.__code__.co_freevars)

<class 'code'>
('c',) ('a', 'b')


Следующий пример. Реализуем счетчик как фунцию-замыкание:

In [ ]:
def make_counter(count=0):
    def counter():
        # сделаем count нелокальным, чтобы можно было его изменять
        nonlocal count
        count += 1
        return count
    return counter

c = make_counter()

In [ ]:
c()     # каждый вызов увеличивает значение count

9

## `__name__`

В программах, которые будут импортированы как модули, часто используется инструкция вида:

In [ ]:
if __name__ == '__main__':
    print('code...')

code...


`__name__` - это специальная переменная, которая получает значение в зависимости от того, как запускается файл с кодом Python. Если файл запускается как основная программа, то переменная `__name__` принимает значение `"__main__"`. Если же файл импортируется из другого файла, то значением переменной `__name__` будет название имя модуля. Поэтому выражение `__name__ == "__main__"` будет возвращать `True` только в том случае, если файл запускается как основная программа, а не модуль.